# Land Cover Classification of RGB Satellite Images using Deep Learning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

# Specify the ZIP file name
zip_file = '/content/drive/MyDrive/data.zip'

# Specify the directory to unzip to (optional, default is current directory)
output_dir = '/content/drive/MyDrive/dataa'

# Unzipping the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"Unzipped files are stored in: {output_dir}")


Unzipped files are stored in: /content/drive/MyDrive/dataa


In [ ]:
#Import needed libraries
import os, shutil
from PIL import Image, ImageOps

#Standard Libraries
import numpy as np
import pandas as pd

# Visualizations
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, classification_report, ConfusionMatrixDisplay

#TensorFlow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from keras import models, layers, optimizers, regularizers
from tensorflow.keras import Model
from tensorflow.data.experimental import cardinality
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense # creates densely connected layer object
from tensorflow.keras.layers import Flatten # takes 2D input and turns into 1D array
from tensorflow.keras.layers import Conv2D # convolution layer
from tensorflow.keras.layers import MaxPooling2D # max pooling layer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#Transfer Learning
from keras.applications import VGG16

### Transfer Learning: VGG16

In [ ]:
train_folder = '/content/drive/MyDrive/dataa/data/split/train'
test_folder = '/content/drive/MyDrive/dataa/data/split/test'

# Normalize images
train16_gen = ImageDataGenerator(rescale=1./255,
                               preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
                               validation_split = 0.125)
test16_gen = ImageDataGenerator(rescale=1./255,
                              preprocessing_function = tf.keras.applications.vgg16.preprocess_input)

#Import data as 70% Train (10% Validation of orginal data set) and 20% Test
vgg16_train_generator = train16_gen.flow_from_directory(train_folder,
                                                class_mode = 'categorical',
                                                subset ='training',
                                                batch_size=128,
                                                shuffle=True,
                                                seed=42)

vgg16_val_generator= train16_gen.flow_from_directory(train_folder,
                                             class_mode= 'categorical',
                                             subset = "validation",
                                             batch_size=128,
                                             shuffle=True,
                                             seed=42)

vgg16_test_generator= test16_gen.flow_from_directory(test_folder,
                                              class_mode= 'categorical',
                                              batch_size=128,
                                              shuffle=False,
                                              seed=42)

Found 18900 images belonging to 10 classes.
Found 2700 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.


In [ ]:
#Loading vgg16 model without the classification layer
vgg16 = VGG16(weights='imagenet',
              include_top=False,
              input_shape=(256, 256, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#Check to ensure there is no prediction layer
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 256, 256, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 256, 256, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 128, 128, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 128, 128, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 64, 64, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 16, 16, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 8, 8, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

After loading the vgg16 model we will now instantiate a sequential model with additional dense and drop out layer

In [ ]:
#Instantiate a Sequential model
vgg16_model = Sequential()

#Input Layer
vgg16_model.add(vgg16)

vgg16_model.add(layers.Flatten())

#Add Dense Layer
vgg16_model.add(layers.Dense(512, activation='relu'))
vgg16_model.add(layers.Dropout(0.2))

#Add Dense Layer
vgg16_model.add(layers.Dense(64, activation='relu'))
vgg16_model.add(layers.Dropout(0.2))

#Output Layer
vgg16_model.add(layers.Dense(10, activation='softmax'))

In [ ]:
#Freeze our VGG16 Layer
vgg16.trainable = False

#Sanity check that VGG16 Layer is frozen
for layer in vgg16_model.layers:
    print(layer.name, layer.trainable)

vgg16 False
flatten True
dense True
dropout True
dense_1 True
dropout_1 True
dense_2 True


In [ ]:
#Compile the model- adam optimizer, categorical_crossentropy loss, and set our metric to accuracy
vgg16_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# print model summary
vgg16_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 8, 8, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      16,777,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │          32,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,525,898 (120.26 MB)

 Trainable params: 16,811,210 (64.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Define Stopping Criteria
valcallback = [EarlyStopping(monitor='val_accuracy', mode='max', verbose = 1, patience = 5),
               ModelCheckpoint(filepath='/content/drive/MyDrive/dataa/try/vgg16_model.keras',
                               monitor='val_accuracy',
                               save_best_only=True,
                               mode='max')]


# Fit the model
vgg16_model_history = vgg16_model.fit(vgg16_train_generator,
                                  epochs= 50,
                                  validation_data = vgg16_val_generator,
                                  callbacks= valcallback,
                                  batch_size=128,
                                  verbose = 1)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


148/148 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.2592 - loss: 2.4164 - val_accuracy: 0.6870 - val_loss: 0.9090
Epoch 2/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.6478 - loss: 0.9844 - val_accuracy: 0.8015 - val_loss: 0.6073
Epoch 3/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 197s 987ms/step - accuracy: 0.7435 - loss: 0.7352 - val_accuracy: 0.8144 - val_loss: 0.5434
Epoch 4/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 147s 953ms/step - accuracy: 0.7808 - loss: 0.6148 - val_accuracy: 0.8200 - val_loss: 0.5180
Epoch 5/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 201s 954ms/step - accuracy: 0.8170 - loss: 0.5285 - val_accuracy: 0.8152 - val_loss: 0.5204
Epoch 6/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 203s 955ms/step - accuracy: 0.8227 - loss: 0.5031 - val_accuracy: 0.8574 - val_loss: 0.4151
Epoch 7/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 201s 953ms/step - accuracy: 0.8486 - loss: 0.4459 - val_accuracy: 0.8663 - val_loss: 0.4079
Epoch 8/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 147s 959ms/step - accuracy: 0.8612 - loss: 0.4047 - v

In [ ]:
# Check loss and accuracy on test data
test_loss, test_acc = vgg16_model.evaluate(vgg16_test_generator, verbose = 1)

print('Test loss: ', test_loss)
print('Test accuracy: ', test_acc)

43/43 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.8959 - loss: 0.3495
Test loss:  0.4533602297306061
Test accuracy:  0.8614814877510071


The VGG16 model seems to work the best of all so far, giving us an even better Test accuracy of 86.1%.